In [1]:
import numpy as np
from matplotlib import pyplot as plt

import time
import datetime
from config import *
from datetime import timedelta
from binance.client import Client
from BinanceFuturesPy.futurespy import Client as cf

In [2]:
client = Client(api_key, api_secret)
clientf = cf(api_key, api_secret)# , testnet=True)

In [5]:
depth = client.futures_coin_order_book(symbol='BTCUSD_PERP', pair="BTCBUSD")
depth

{'lastUpdateId': 212303541999,
 'E': 1631964892757,
 'T': 1631964892749,
 'symbol': 'BTCUSD_PERP',
 'pair': 'BTCUSD',
 'bids': [['48632.1', '7881'],
  ['48631.9', '505'],
  ['48631.8', '824'],
  ['48631.6', '71'],
  ['48631.4', '122'],
  ['48631.2', '126'],
  ['48630.5', '120'],
  ['48630.0', '1'],
  ['48629.9', '100'],
  ['48629.1', '261'],
  ['48628.8', '107'],
  ['48628.7', '176'],
  ['48628.3', '146'],
  ['48628.0', '10'],
  ['48627.5', '150'],
  ['48627.2', '457'],
  ['48626.9', '11'],
  ['48626.6', '170'],
  ['48626.4', '2'],
  ['48625.9', '300'],
  ['48625.7', '307'],
  ['48625.5', '40'],
  ['48625.4', '40'],
  ['48625.3', '40'],
  ['48625.2', '136'],
  ['48624.9', '60'],
  ['48624.7', '175'],
  ['48624.3', '172'],
  ['48624.2', '100'],
  ['48623.9', '1'],
  ['48623.2', '130'],
  ['48623.0', '1160'],
  ['48622.4', '1'],
  ['48622.3', '159'],
  ['48622.0', '440'],
  ['48621.9', '40'],
  ['48621.7', '286'],
  ['48621.6', '416'],
  ['48621.5', '10'],
  ['48621.3', '186'],
  ['48620

In [17]:
bb = depth['bids']
b_p = np.array(bb)[:, 0].astype(np.float64)
b_p[0]

48632.1

In [18]:
aa = depth['asks']
a_p = np.array(aa)[:, 0].astype(np.float64)
a_p[0]

48632.2

In [19]:
price = (b_p[0]+a_p[0])/2
price

48632.149999999994

In [3]:
# clientf.balance()

In [4]:
# clientf.change_leverage?

In [5]:
# %%timeit
# client.futures_account()

In [6]:
# acc = client.futures_account()
# del acc['assets']
# del acc['positions']
# acc

In [7]:
def depth_book(depth):
    bid = np.array(depth['bids']).astype(np.float64)
    bid_sum = bid.sum(axis=0) # t_value | orders

    ask = np.array(depth['asks']).astype(np.float64)
    ask_sum = ask.sum(axis=0) # t_value | orders
    
    b_price = bid[:, 0][0]
    a_price = ask[:, 0][0]
    price = (b_price+a_price)/2
    
    # bid ask price histograms
    bid_hist = np.histogram(bid[:, 0])
    ask_hist = np.histogram(ask[:, 0])
    
    # total volumen
    vol_bid = bid_sum[0]*bid_sum[1]
    vol_ask = ask_sum[0]*ask_sum[1]
    vol_total = vol_bid+vol_ask # total vol
    diff_vol = vol_bid-vol_ask # direction
    
    # porcentaje representativo
    p_bid = (vol_bid/vol_total)*100
    p_ask = (vol_ask/vol_total)*100
    p_dif = (abs(diff_vol)/vol_total)*100

    # std num orders
    std_order_bid = round(np.std(bid_hist[0]), 3)
    std_order_ask = round(np.std(ask_hist[0]), 3)
    
#     # str %
#     p_bid_str = "{:.3f}".format(p_bid)
#     p_ask_str = "{:.3f}".format(p_ask)
#     p_dif_str = "{:.3f}".format(p_dif)
    
#     fig = plt.figure(figsize=(8, 5))
#     # plt.title(f"price: ${round(float(price), 2)}")
#     plt.bar(["bid", "ask"], [std_order_bid, std_order_ask], color=['green', 'red'])
#     # plt.hist(bid[:, 0], color='green')
#     # plt.hist(ask[:, 0], color='red')
#     plt.title(f'bid {p_bid_str}% std_bid = {std_order_bid}\nask {p_ask_str}% std_ask = {std_order_ask}')
#     plt.ylim([0, 40])
#     plt.grid()
#     plt.show()
    return std_order_bid, std_order_ask, price

In [8]:
def buy_order():
    order = clientf.new_order(
                        # symbol = 'BTCUSDT',
                        symbol = 'BTCBUSD',
                        side = 'BUY',
                        orderType = 'MARKET',
                        quantity = 1e-3,
                    )
    return order

def sell_order():
    order = clientf.new_order(
                        # symbol = 'BTCUSDT',
                        symbol = w'BTCBUSD',
                        side = 'SELL',
                        orderType = 'MARKET',
                        quantity = 1e-3,
                    )
    return order

In [9]:
# def buy_order():
#     return 1

# def sell_order():
#     return 0

In [ ]:
i_3 = 0
activo = 0
on = 0
side = ""
lado1 = 0
lado2 = 0

velas = 0
V = []

sl = []
record = []

while True:
    time.sleep(2)
    # price = client.futures_ticker(symbol='BTCUSDT')['lastPrice']
    try:
        # price = 48327.6
        depth = client.futures_coin_order_book(symbol='BTCUSD_PERP', pair="BTCBUSD")
        # price = client.futures_coin_mark_price(symbol='BTCUSD_PERP')
        b, a, price = depth_book(depth)
        
        diff = abs(b-a)
        if diff>=5 and on==0:
            i_3+=1
        elif diff>=6 and on==1:
            i_3+=1
        else:
            diff=0
        
        if i_3>=4 and activo==0:
            activo=1
        elif i_3>=2 and on==1:
            activo=0
#         else:
#             activo=0
#             # side=""
        
        if b>a:
            lado2 = 1
        elif a>b:
            lado2 = -1
            
        ### 1~
        if activo==1 and on==0:
                if b>a:
                    ### venta
                    order = sell_order()
                    side = "sell"
                    lado1 = 1
                else:
                    ### compra
                    order = buy_order()
                    side = 'buy'
                    lado1 = -1
                on=1
        elif activo==0 and on==1:
            if side=="sell":
                order = buy_order()
            else:
                order = sell_order()
            on=0
            side=""
        
        elif activo==1 and on==1 and lado1!=lado2 and side!="":
            if side=="sell":
                order = buy_order()
            elif side=="buy":
                order = sell_order()
            activo = 0
            on = 0
            side = ''
            
        ### 1.5
        if on==1:
            recorde.append(price)
            
        ### 2
        # acc = client.futures_account()['totalUnrealizedProfit']
        # acc = client.futures_account()['assets'][2]['unrealizedProfit']
        
        if len(record)>1:
            acc = (record[-1]-record[-2])/2
        else:
            acc = 0
        
        # acc = float(acc)
        # print(acc)
        cond = acc<-0.002
        if cond:
            if side=="sell":
                order = buy_order()
            elif side=="buy":
                order = sell_order()
            activo = 0
            on = 0
            side = ""
            
        if acc>0:
            acc2 = acc*.75
            sl.append(acc2)
        if len(sl)>0:
            stop = max(sl)
            if acc<stop:
                if side=="sell":
                    order = buy_order()
                elif side=="buy":
                    order = sell_order()
                activo = 0
                on = 0
                side = ""
        
    # except requests.exceptions.ConnectTimeout:
    except SyntaxError:
        #print(".", end=" ")
        print("timeout")
        # pass
        if activo==1 and on==1:
            if side=="sell":
                order = buy_order()
            elif side=="buy":
                order = sell_order()
            else:
                pass
            activo = 0
            on = 0
            side = ''
        time.sleep(15)
    # time.sleep(60)

0.0
0.0
0.0
0.0
-0.00709003
0.0
0.01238516
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
T = [[27.236, 21.288], [27.9, 22.154], [28.365, 22.186], [27.814, 22.19], [30.437, 17.983], [30.206, 18.281], [30.087, 18.199], [30.285, 19.839], [30.285, 19.277], [30.516, 19.105], [29.99, 16.223], [30.358, 17.771], [30.656, 17.652], [30.958, 17.709], [30.659, 17.395], [29.923, 16.834], [30.126, 17.652], [31.203, 17.286], [30.816, 17.669], [31.215, 19.915], [29.776, 17.838], [29.213, 19.298], [29.557, 19.173], [30.007, 19.053], [24.544, 19.042], [24.121, 20.0], [23.58, 19.126], [24.731, 17.821], [23.533, 17.624], [24.69, 19.677], [24.519, 19.657], [29.421, 20.616], [24.29, 16.971], [24.245, 19.975], [26.612, 21.312], [26.623, 20.298], [27.968, 19.662], [28.182, 19.774], [28.057, 19.416], [25.052, 21.767], [25.263, 21.157], [24.739, 21.014], [23.854, 21.024], [28.646, 23.186], [25.632, 20.659], [26.104, 18.122], [25.946, 19.94], [27.514, 16.733], [26.773, 17.933], [27.885, 16.254], [27.745, 15.975], [30.013, 17.395], [30.64, 16.081], [29.96, 16.26], [30.961, 16.395], [28.622, 16.559], [28.962, 17.17], [28.921, 16.703], [28.114, 18.569], [28.834, 19.396], [28.376, 18.401], [28.439, 18.989], [28.824, 18.466], [27.59, 18.281], [29.103, 18.687], [29.028, 18.639], [22.79, 12.562], [21.735, 12.091], [22.226, 12.442], [22.952, 12.767], [22.325, 13.015], [15.582, 19.396]]